<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #7: Transfer Object Detection models to Zenodo</h1>
<h3 align="right">Written by KSO Team</h3>

# 1. Set up and requirements
### Install and import Python packages

In [ ]:
from IPython.display import clear_output

try:
    import google.colab
    import os

    IN_COLAB = True
    print("Running in Colab...")

    # Clone repo
    !git clone --recurse-submodules https://github.com/ocean-data-factory-sweden/koster_yolov4.git
    !pip install -q --upgrade pip
    !pip install -qr koster_yolov4/requirements.txt

    # Fix libmagic issue
    !apt-get -qq update && apt-get -qq install -y libmagic-dev > /dev/null

    # Replace upsampling script with custom version
    os.chdir("koster_yolov4/tutorials")
    !mv ../src/upsampling.py /usr/local/lib/python3.7/dist-packages/torch/nn/modules/upsampling.py

    # Replace nearest neighbours script with custom version (due to relative path issue)
    !cp ../src/nn_matching.py ../yolov5_tracker/strong_sort/sort/nn_matching.py

    # Enable external widgets
    from google.colab import output

    output.enable_custom_widget_manager()

    # Ensure widgets are shown properly
    !jupyter nbextension enable --user --py widgetsnbextension
    !jupyter nbextension enable --user --py jupyter_bbox_widget

    print("All packages are installed and ready to go!")
    try:
        clear_output()
        print("All packages are installed and ready to go!")
    except:
        clear_output()
        print("There have been some issues installing the packages!")
except:
    IN_COLAB = False
    import sys
    import pkgutil

    if pkgutil.find_loader("torch") is not None:
        !pip install -q --upgrade pip
        !pip install -q torch==1.8.0 torchvision==0.9.0
    # Replace nearest neighbours script with custom version (due to relative path issue)
    !cp ../src/nn_matching.py ../yolov5_tracker/strong_sort/sort/nn_matching.py

    # Ensure widgets are shown properly
    !jupyter nbextension enable --user --py widgetsnbextension
    !jupyter nbextension enable --user --py jupyter_bbox_widget
    clear_output()
    print("Running locally... you're good to go!")

In [ ]:
# Set the directory of the libraries
import sys, os
from pathlib import Path

sys.path.append("..")

# Enables testing changes in utils
%load_ext autoreload
%autoreload 2

# Set to display dataframes as interactive tables
from itables import init_notebook_mode

init_notebook_mode(all_interactive=True)

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.project_utils as p_utils
import kso_utils.t6_utils as t6
import kso_utils.t7_utils as t7
import kso_utils.zenodo_utils as zenodo_utils
from kso_utils.project import ProjectProcessor, MLProjectProcessor

clear_output()
print("Packages loaded successfully")

# 2. Find model and add to Zenodo

### Choose your project

In [ ]:
project_name = t_utils.choose_project()

In [ ]:
project = p_utils.find_project(project_name=project_name.value)

In [ ]:
# Initialise pp
pp = ProjectProcessor(project)

In [ ]:
# Initialise mlp
mlp = MLProjectProcessor(pp)

### Choose model

In [ ]:
model = t6.choose_model(project_name.value)

### Download model

In [ ]:
download_dir = t_utils.choose_folder(".", "downloaded model")

In [ ]:
artifact_dir = mlp.get_model(model.value, download_dir.selected)

# 2. ZENODO Upload

### Provide access token to your account (found in API settings by going to your Zenodo profile and clicking on Applications)

In [ ]:
ACCESS_TOKEN = ""

### Get bucket url for uploading files

In [ ]:
depo_id, bucket_url = zenodo_utils.get_zenodo_id_bucket(ACCESS_TOKEN)

### Find archive for upload to Zenodo

In [ ]:
archive_dir = t_utils.choose_folder(".", "archive for upload")

### Add archive to upload

In [ ]:
zenodo_utils.add_file_to_zenodo_upload(
    ACCESS_TOKEN,
    bucket_url,
    [f for f in Path(artifact_dir).iterdir() if f.is_file() and ".pt" in str(f)][-1],
)

### Add dataset metadata to upload (title, description and authors with their organisations)

In [ ]:
upload_title = t7.choose_text("title")

In [ ]:
upload_description = t7.choose_text("description")

In [ ]:
authors = t7.WidgetMaker()
authors

In [ ]:
authors.author_dict

In [ ]:
zenodo_utils.add_metadata_zenodo_upload(
    ACCESS_TOKEN,
    depo_id,
    upload_title.value,
    upload_description.value,
    authors.author_dict,
)

In [ ]:
# END